# Final Assignment, Part I

In [14]:
%matplotlib widget

from pathlib import Path

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Thanks to `ImportanceOfBeingErnest` from https://stackoverflow.com/questions/47404653/pandas-0-21-0-timestamp-compatibility-issue-with-matplotlib
pd.plotting.register_matplotlib_converters()

input_dirpath = Path('../data/raw/wisdm/raw')
processed_dirpath = Path('../data/processed/wisdm/')
ref_dirpath = Path('../references')

activity_key_path = ref_dirpath / 'wisdm_activity_key.txt'

DEBUG=True
def release_plt_mem():
    if not DEBUG:
        return
    try:
        # release memory if needed
        plt.close('all')
    except:
        pass


In [7]:
activity_key = {v: k for k, v in [line.strip().split(' = ') for line in activity_key_path.read_text().strip().split('\n')]}
# '| Keys | Activity |\n|:-:|:-:|' + '\n'.join(['|{}|{}|'.format(k, v) for k, v in activity_key.items()])

# Data Information
> from Documentation

| Keys | Activity |
|:-:|:-:|
|A|walking|
|B|jogging|
|C|stairs|
|D|sitting|
|E|standing|
|F|typing|
|G|teeth|
|H|soup|
|I|chips|
|J|pasta|
|K|drinking|
|L|sandwich|
|M|kicking|
|O|catch|
|P|dribbling|
|Q|writing|
|R|clapping|
|S|folding|

| Field name | Type | Description | Range |
|:--|:--|:--|:--|
| Subject-id |  Symbolic numeric identififier. | Uniquely identifyies the subject. | 1600-1650. |
| Activity code | Symbolic single letter. | Identifies a specific activity as listed in Table 2. | A-S (no “N” value) |
| Timestamp  | Integer. | Linux time | >= 1-1-1970 00\:00\:00.0+00\:00 |
| x | Numeric: real.  | Sensor value for x axis. | May be positive or negative. |
| y | Numeric: real.  | Sensor value for y axis. | May be positive or negative. |
| z | Numeric: real.  | Sensor value for z axis. | May be positive or negative. |

In [24]:
# process data for ease of access - only run once
            
def concat_subject_data_by_device_type():
    """ Construct files containing all subjects and activities for each device and sensor type. """
    import zipfile

    df_dict = {}
    # from data documentation
    headers = ['subject_id', 'activity_code', 'timestamp', 'x','y','z']

    for f in input_dirpath.glob('**/*.zip'):
        unzip_dirpath = f.parent / f.stem
        if not unzip_dirpath.exists():
            with zipfile.ZipFile(str(f),'r') as fd:
                fd.extractall(str(f.parent))

    for f in input_dirpath.glob('raw/**/*.txt'):
        if not any([name in str(f) for name in ['watch', 'phone']]):
            continue

        split_path = f.parts
        device_type, sensor_type  = split_path[-3:-1]
        data_src = '{}_{}'.format(device_type, sensor_type)

        filename = output_dirpath / 'wisdm_{}.csv'.format(data_src)
        if filename.exists():
            continue

        new_df = pd.read_csv(f, names=headers, parse_dates=['timestamp'], infer_datetime_format=False)
        new_df['timestamp'] = [pd.to_datetime(int(ts), utc=True) for ts in new_df['timestamp']]
        
        # remove idiosyncratic endline symbol
        new_df.iloc[:,-1] = new_df.iloc[:,-1].str.replace(';', '')
        
        if dfs.get(data_src) is None:
            df_dict[data_src] = new_df
        else:
            df_dict[data_src] = dfs[data_src].append(new_df)

    for data_src, df in df_dict.items():
        df.to_csv(output_dirpath / 'wisdm_{}.csv'.format(data_src), index=False)

    return df_dict
    
        
# this doesn'n need to be called if data is already in the /data/processed/wisdm directory  
dfs = concat_subject_data_by_device_type()

In [218]:
# If the cell above is not run, this will recreate a dictionary with the same structure
# This can take awhile as the files are large
dfs = {f.stem.replace('wisdm_', ''): pd.read_csv(f, parse_dates=['timestamp']) for f in processed_dirpath.glob('wisdm_*.csv')}
dfs.keys()

dict_keys(['phone_accel', 'phone_gyro', 'watch_accel', 'watch_gyro'])

In [ ]:
display(dfs.get('phone_accel').head())
display(dfs.get('phone_accel').tail())

In [ ]:
display(dfs.get('phone_accel').dtypes)
display(dfs.get('watch_accel').dtypes)

In [ ]:
df = dfs.get('phone_accel').groupby('activity_code')
np.median(df.get_group('A')[['x', 'y', 'z']], axis=0)
# df[(df['subject_id']==1600) & (df['activity_code']=='A')]#.cumsum(axis=0)

From page 2 of the documentation:
>For the accelerometer sensor, the units are
$m/s^2$, ... Note that the force of gravity on Earth,
which affects the accelerometer readings, is $9.8m/s^2$.

Note: *y* axis is Vertical, not *z*. Why is this using an image processing system?

| Axis | Direction |
|:-:|:-:|
|x|Forward/Backward|
|y|Up/Down|
|z|Left/Right|

In [ ]:
df = dfs.get('phone_accel').groupby(['subject_id', 'activity_code'])
df_sub = df.get_group((1600, 'A'))
display(df_sub['y'].head())
display(df_sub['y'].mean())

## Tare To Origin / Mean Functions
Used to set the first index or the mean as the origin, substracting it from subsequent values.
This helps remove artifacts which bias the data when comparing sets

In [166]:
def standardize_to_origin(df_):
    """  Adjust values using initial index as `origin`  (otherwise inital bias takes over during cumulative summation). """
    return df_.assign(**{col: df_[col] - df_[col].iloc[0] for col in ['x', 'y', 'z']})

def standardize_to_mean(df_):
    """ Adjust values using mean as `origin` (otherwise inital bias takes over during cumulative summation). """
    return df_.assign(**{col: df_[col] - df_[col].mean() for col in ['x', 'y', 'z']})

In [194]:
df = dfs.get('phone_accel').groupby(['subject_id', 'activity_code'])
df_sub = df.get_group((1600, 'A'))

for name, fn in [('standardize_to_origin', tastandardizeo_origin), ('standardize_to_mean', standardize_to_mean)]:
    df_sub_adj = fn(df_sub)
    # display(df_sub_adj.nlargest(5, columns='y').append(df_sub_adj.nsmallest(5, columns='y')))
    print('\n>>>\n>>> ' + name.replace('_', ' ').capitalize() + ' <<<\n>>>')
    display(df_sub_adj.sort_values(by='y'))
    print('\n>   Mean Values after Tare')
    display(pd.DataFrame(np.round(df_sub_adj.mean(), decimals=6)).T)

# df_sub_adj = tare_to_mean(df_sub)
# # display(df_sub_adj.nlargest(5, columns='y').append(df_sub_adj.nsmallest(5, columns='y')))
# display(df_sub_adj.sort_values(by='y'))
# display(pd.DataFrame(np.round(df_sub_adj.mean(), decimals=5)))


>>>
>>> Tare to origin <<<
>>>


,subject_id,activity_code,timestamp,x,y,z
689,1600,A,1970-01-03 22:04:02.360736882+00:00,-2.073883,-7.831177,-0.571075
1207,1600,A,1970-01-03 22:04:28.444107868+00:00,-0.152618,-7.586121,-1.677689
1553,1600,A,1970-01-03 22:04:45.866598547+00:00,-3.946213,-7.571045,-1.735626
287,1600,A,1970-01-03 22:03:42.118420721+00:00,-1.213669,-7.570648,-0.835968
408,1600,A,1970-01-03 22:03:48.211246563+00:00,-0.835831,-7.429840,-1.566391
...,...,...,...,...,...,...
2890,1600,A,1970-01-03 22:05:53.189886795+00:00,-3.262238,10.316971,1.221466
971,1600,A,1970-01-03 22:04:16.560547824+00:00,-5.989212,10.449936,6.581345
1638,1600,A,1970-01-03 22:04:50.146667980+00:00,-1.816086,10.619064,3.497986
2912,1600,A,1970-01-03 22:05:54.297674881+00:00,0.173492,10.664429,-1.734695



>   Mean Values after Tare


,subject_id,x,y,z
0,1600.0,-1.493528,0.757595,-0.63885



>>>
>>> Tare to mean <<<
>>>


,subject_id,activity_code,timestamp,x,y,z
689,1600,A,1970-01-03 22:04:02.360736882+00:00,-0.580355,-8.588772,0.067775
1207,1600,A,1970-01-03 22:04:28.444107868+00:00,1.340909,-8.343716,-1.038838
1553,1600,A,1970-01-03 22:04:45.866598547+00:00,-2.452685,-8.328640,-1.096776
287,1600,A,1970-01-03 22:03:42.118420721+00:00,0.279859,-8.328244,-0.197118
408,1600,A,1970-01-03 22:03:48.211246563+00:00,0.657697,-8.187435,-0.927541
...,...,...,...,...,...,...
2890,1600,A,1970-01-03 22:05:53.189886795+00:00,-1.768710,9.559376,1.860317
971,1600,A,1970-01-03 22:04:16.560547824+00:00,-4.495684,9.692341,7.220195
1638,1600,A,1970-01-03 22:04:50.146667980+00:00,-0.322558,9.861469,4.136836
2912,1600,A,1970-01-03 22:05:54.297674881+00:00,1.667020,9.906834,-1.095845



>   Mean Values after Tare


,subject_id,x,y,z
0,1600.0,-0.0,0.0,0.0


### Note
$y$ should be nearly $0$ with gravity taken into account. `Tare To Mean` works well.

## Coordinate Summation
If acceleration data is treaded as *instantaneous* acceleration ($\bf{a}$), taking the cumulative sum as a *knockoff* version of an integral over $\bf{a}$ gives us a velocity ($\bf{v}$).

This is probably better stated, informally speaking, as a *pseudo*-velocity. 

**NOTE**: Technically, this should be divided by the frame rate (~25fps). For exploratory purposes, this probably isn't necessary to get the gist of the space.

In [109]:
def coord_summation(df_, combine=False):
    """ For cumulatively sum coordinate vectors across time"""
    return df_.assign(**{'{}{}'.format(col, '_sum' if combine else ''): vals.cumsum(axis=0) for col, vals in df_[['x', 'y', 'z']].items()})

In [ ]:
df_sub_adj = standardize_to_mean(df_sub)
df_sub_sum = coord_summation(df_sub_adj)
df_sub_samp = df_sub_adj.iloc[:1000]
df_sub_sum = df_sub_sum.iloc[:1000]

# display({col: df_sub_sum[['x', 'y', 'z']].cumsum(axis=0) for col, vals in df_sub_sum[['x', 'y', 'z']].items()})

In [ ]:
mins = df_sub_sum[['x','y','z']].aggregate('min')
maxs = df_sub_sum[['x','y','z']].aggregate('max')

mins.name = 'minimums'
maxs.name = 'maximums'

display(mins)
display(maxs)

total_min = mins.min()
total_max = maxs.max()

abs_minmax = np.max(np.abs([total_min, total_max]))

print()
print('Total Minimum: {:.04f}'.format(total_min))
print('Total Maximum: {:.04f}'.format(total_max))
print('Absolute MinMax: {:.04f}'.format(abs_minmax))

In [7]:
def get_abs_minmax(df_):
    total_min = df_[['x','y','z']].aggregate('min').min()
    total_max = df_[['x','y','z']].aggregate('max').max()
    return np.max(np.abs([total_min, total_max]))

# Acceleration Analysis

> in $m/s^2$


In [8]:
release_plt_mem()

df_sub_sum = coord_summation(df_sub_adj)
df_sub_samp = df_sub_adj.iloc[:1000]
df_sub_sum = df_sub_sum.iloc[:1000]

abs_minmax = get_abs_minmax(df_sub_sum) + 100

""" Start Plot """
fig, axs = plt.subplots(3,1,sharex=True)
fig.suptitle('Phone Accelerometer for Subject #1600 - Walking')
ax_vert, ax_lng, ax_lat = axs

ax_vert.plot(df_sub_samp['timestamp'], df_sub_sum['y'])
ax_lng.plot(df_sub_samp['timestamp'], df_sub_sum['x'])
ax_lat.plot(df_sub_samp['timestamp'], df_sub_sum['z'])

for ax in axs:
    ax.set_ylim(-(abs_minmax), abs_minmax)

ax_vert.set_ylabel('Y (Vertical Axis)')
ax_lng.set_ylabel('X (Longitudnal Axis)')
ax_lat.set_ylabel('Z (lateral Axis)')

ax_vert.grid()
ax_lng.grid()
ax_lat.grid()

ax_lat.set_xlabel('Time')

fig.canvas.layout.width = '1600px'
fig.canvas.layout.height = '1200px'

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
df = dfs.get('phone_accel')
df = df[(df['subject_id'] == 1600)]

def get_abs_subject_max(df_):
    return np.max([get_abs_minmax(coord_summation(standardize_$1(df_[(df_['activity_code'] == key)]))[['x', 'y', 'z']]) for key, activity in activity_key.items()])

get_abs_subject_max(df)

1037.0013180442156

In [40]:
from mpl_toolkits.mplot3d import Axes3D

def plot_activity_accel_over_time(df_, subject_id=1600, release_mem=False):
    if release_mem:
        release_plt_mem()
    df_subject = df_[(df_['subject_id']==subject_id)]

    fig_ = plt.figure()
    ax = fig_.add_subplot(111, projection='3d')

    subject_abs_max = get_abs_subject_max(df_subject)

    for key, activity in activity_key.items():
        df_subject_activity = standardize_$1(df_subject[(df_subject['activity_code']==key)][['x', 'y', 'z']])
        df_subject_sum = coord_summation(df_subject_activity)

        ax.plot(df_subject_sum['x'], df_subject_sum['z'], df_subject_sum['y'], label=activity)

        ax.set_xlabel('Longitudnal')
        ax.set_ylabel('Lateral')
        ax.set_zlabel('Vertical')

        ax.set_xlim(-subject_abs_max, subject_abs_max)
        ax.set_ylim(-subject_abs_max, subject_abs_max)
        ax.set_zlim(-subject_abs_max, subject_abs_max)

    fig_.legend(title='Activity')
    fig_.suptitle('Subject #1600\'s Summed Acceleration per Activity in 3D-Space Over Time.')
    
    return fig_

In [40]:
fig = plot_activity_accel_over_time(dfs.get('phone_accel'), 1600)
fig.canvas.layout.width = '1200px'
fig.canvas.layout.height = '1200px'

fig.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [41]:
fig = plot_activity_accel_over_time(dfs.get('watch_accel'), subject_id=1620)

fig.canvas.layout.width = '1200px'
fig.canvas.layout.height = '1200px'

fig.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Gyroscopic Analysis

>(for) the gyroscope sensor, the units are $\theta/s$

Not sure about this wrt this data:

| Axis | Envelope |
|:-:|:-:|
|x|roll|
|y|pitch|
|z|yaw|

**NOTE**: It is unclear if these data run into gimble-lock situations. Given the activity domain, it is doubtful this would be an issue.

In [159]:
df_watch_gyro = dfs.get('watch_gyro')
df_phone_gyro = dfs.get('phone_gyro')

subject = 1650

df_watch_gyro = df_watch_gyro[df_watch_gyro['subject_id'] == subject]
df_phone_gyro = df_phone_gyro[df_phone_gyro['subject_id'] == subject]

max_frames = 200

# display(df_watch_gyro.head())
release_plt_mem()
# https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.subplots.html?highlight=subplots#matplotlib.pyplot.subplots
fig, axs = plt.subplots(1,2, subplot_kw=dict(polar=True))

ax_watch, ax_phone = axs

#
# Watch
#

# Activity D: Sitting
df = standardize_$1(df_watch_gyro[df_watch_gyro['activity_code']=='D'].iloc[:max_frames])
ax_watch.plot(df.index, df['x'], label=activity_key.get('D'))

# Activity Q: Writing
df = standardize_$1(df_watch_gyro[df_watch_gyro['activity_code']=='Q'].iloc[:max_frames])
ax_watch.plot(df.index, df['x'], label=activity_key.get('Q'))

#
# Phone
#

# Activity D: Sitting
df = standardize_$1(df_phone_gyro[df_phone_gyro['activity_code']=='D'].iloc[:max_frames])
ax_phone.plot(df.index, df['x'], label=activity_key.get('D'))

# Activity Q: Writing
df = standardize_$1(df_phone_gyro[df_phone_gyro['activity_code']=='Q'].iloc[:max_frames])
ax_phone.plot(df.index, df['x'], label=activity_key.get('Q'))


for ax in axs:
    ax.set_rlim(ax.get_rmin(), 8)
    ax.set_xlabel('Gyroscope Rotation (in radians)')
    ax.text(np.radians(ax.get_rlabel_position()+10), ax.get_rmax() - 2, 'Time', rotation=ax.get_rlabel_position() + 5, ha='center',va='center')

fig.legend(title='Activity')    
fig.canvas.layout.width = '1200px'
fig.canvas.layout.height = '1200px'

fig.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [164]:
df_watch_gyro = dfs.get('watch_gyro')
df_phone_gyro = dfs.get('phone_gyro')

subject = 1650

df_watch_gyro = df_watch_gyro[df_watch_gyro['subject_id'] == subject]
df_phone_gyro = df_phone_gyro[df_phone_gyro['subject_id'] == subject]


# display(df_watch_gyro.head())
release_plt_mem()
# https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.subplots.html?highlight=subplots#matplotlib.pyplot.subplots
fig, axs = plt.subplots(2,1)

ax_watch, ax_phone = axs

#
# Watch
#

# Watch Activity A: Walking
df = coord_summation(standardize_$1(df_watch_gyro[df_watch_gyro['activity_code']=='A'].reset_index(drop=True)))
ax_watch.plot(df['x'].index, df['x'], label='{}'.format(activity_key.get('A')))

# Watch Activity D: Sitting
df = coord_summation(standardize_$1(df_watch_gyro[df_watch_gyro['activity_code']=='D'].reset_index(drop=True)))
ax_watch.plot(df['x'].index, df['x'], label='{}'.format(activity_key.get('D')))

# watch Activity Q: Writing
df = coord_summation(standardize_$1(df_watch_gyro[df_watch_gyro['activity_code']=='Q'].reset_index(drop=True)))
ax_watch.plot(df['x'].index, df['x'], label='{}'.format(activity_key.get('Q')))
ax_watch.set_title('Watch Gyroscope')

#
# Phone
#

# Phone Activity A: Walking
df = coord_summation(standardize_$1(df_phone_gyro[df_phone_gyro['activity_code']=='A'].reset_index(drop=True)))
ax_phone.plot(df['x'].index, df['x'], label='{}'.format(activity_key.get('A')))

# Phone Activity D: Sitting
df = coord_summation(standardize_$1(df_phone_gyro[df_phone_gyro['activity_code']=='D'].reset_index(drop=True)))
ax_phone.plot(df['x'].index, df['x'], label='{}'.format(activity_key.get('D')))

# watch Activity Q: Writing
df = coord_summation(standardize_$1(df_phone_gyro[df_phone_gyro['activity_code']=='Q'].reset_index(drop=True)))
ax_phone.plot(df['x'].index, df['x'], label='{}'.format(activity_key.get('Q')))
ax_phone.set_title('Phone Gyroscope')

for ax in axs:
    ax.grid()
    ax.set_xlabel('Time')
    ax.set_ylabel('Cumulative Radians per Second')
    ax.legend()

   
fig.canvas.layout.width = '1200px'
fig.canvas.layout.height = '1200px'

fig.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [217]:
df_watch_accel = dfs.get('watch_accel')
df_phone_accel = dfs.get('phone_accel')

# display(len(set(df_watch_accel['activity_code'])))
display((set(df_phone_accel['subject_id'])))

# subject = 1650
# activity_code = 'A'

# df_watch_accel = coord_summation(standardize_$1(df_watch_accel[(df_watch_accel['subject_id'] == subject) & (df_watch_accel['activity_code'] == activity_code)]), True).reset_index(drop=True)
# df_phone_accel = coord_summation(standardize_$1(df_phone_accel[(df_phone_accel['subject_id'] == subject) & (df_phone_accel['activity_code'] == activity_code)]), True).reset_index(drop=True)

# display(df_watch_accel)
# display(df_phone_accel)

# fig, axs = plt.subplots(1,1)
# ax1 = axs

# ax1.plot(df_watch_accel['y_sum'])
# ax1.plot(df_phone_accel['y_sum'])

# fig.show()

{1600}

### Gyroscope + Accelerometer
To get the actual heading of the device, the coordinates need to be rotated. A rotation matrix might be of use (taken from [Wolfram-Alpha - Rotation Matrix](http://mathworld.wolfram.com/RotationMatrix.html))
<br/><br/>
Rotation around the x-axis, where $\alpha = \text{x_gyro}$:
$$
R_x(\alpha) = 
\begin{bmatrix}
    1 & 0 & 0 \\
    0 & cos\;\alpha & -sin\;\alpha \\
    0 & sin\;\alpha & cos\;\alpha \\
\end{bmatrix}
\\[2em]
$$
Rotation around the y-axis, where $\beta = \text{y_gyro}$:
$$
R_y(\beta) = \begin{bmatrix}
cos\;\beta & 0 & sin\;\beta \\
0 & 1 & 0 \\
-sin\;\beta & 0 & cos\;\beta \\
\end{bmatrix}
\\[2em]
$$
Rotation around the z-axis, where $\gamma = \text{z_gyro}$
$$
R_z(\gamma) = \begin{bmatrix}
cos\;\gamma & -sin\;\gamma & 0 \\
sin\;\gamma & cos\;\gamma & 0 \\
0 & 1 & 0 \\
\end{bmatrix}
$$

These can be combined to create ... (more to come)